In [1]:
!pip install accelerate evaluate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import torch
import torch.nn as nn
import transformers
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import torch.optim as optim
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from accelerate import Accelerator
from tqdm.auto import tqdm
from evaluate import load

In [5]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
base_model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=6, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model_4bit = prepare_model_for_kbit_training(base_model)

In [8]:
config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

In [9]:
model = get_peft_model(model_4bit, config)

In [10]:
accelerator = Accelerator(mixed_precision="bf16")
device = accelerator.device

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [12]:
class ToxicCommentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float)
        return item

In [13]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X = train_df["comment_text"]
y = train_df[["toxic",
        "severe_toxic",
        "obscene",
        "threat",
        "insult",
        "identity_hate"]]

In [14]:
def tokenize_text(texts, max_length=128):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )

# Tokenize the input text
tokenized_train_texts = tokenize_text(X)
tokenized_test_texts = tokenize_text(test_df["comment_text"])

In [21]:
train_dataset = ToxicCommentDataset(tokenized_train_texts, y)
test_dataset = ToxicCommentDataset(
    tokenized_test_texts,
    test_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]],
)
batch_size = 128
train_data_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
test_data_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

In [22]:
# configuring hf accelerate
train_data_loader, test_data_loader, model, optimizer = accelerator.prepare(
    train_data_loader, test_data_loader, model, optimizer
)

In [23]:
epochs = 3
training_steps = epochs * len(train_data_loader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=training_steps,
)

In [27]:
progress_bar = tqdm(range(training_steps))

  0%|          | 0/624 [00:00<?, ?it/s]

In [28]:
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    model.train()
    for batch in train_data_loader:
        outputs = model(**batch)
        loss = outputs.loss

        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)

        total_loss += loss.item()

    avg_loss = total_loss / len(train_data_loader)
    print(f"Epoch {epoch + 1} - Average Loss: {avg_loss:.4f}")

    # Evaluate the model
    model.eval()
    all_predictions = []
    all_labels = []
    for batch in test_data_loader:
        with torch.no_grad():
            outputs = model(**batch)
        predictions = (torch.sigmoid(outputs.logits) > 0.5).int()
        all_predictions.append(predictions)
        all_labels.append(batch["labels"])

    all_predictions = torch.cat(all_predictions, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    accuracies = (all_predictions == all_labels).float().mean(dim=0)
    avg_accuracy = accuracies.mean().item()

    print(f"Average Accuracy across all labels: {avg_accuracy:.4f}")
    label_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    for i, acc in enumerate(accuracies):
        print(f"Accuracy for {label_names[i]}: {acc:.4f}")

progress_bar.close()

Epoch 1/3
Epoch 1 - Average Loss: 0.0858
Average Accuracy across all labels: 0.9748
Accuracy for toxic: 0.9490
Accuracy for severe_toxic: 0.9930
Accuracy for obscene: 0.9600
Accuracy for threat: 0.9970
Accuracy for insult: 0.9580
Accuracy for identity_hate: 0.9920
Epoch 2/3
Epoch 2 - Average Loss: 0.0685
Average Accuracy across all labels: 0.9813
Accuracy for toxic: 0.9520
Accuracy for severe_toxic: 0.9920
Accuracy for obscene: 0.9810
Accuracy for threat: 0.9970
Accuracy for insult: 0.9740
Accuracy for identity_hate: 0.9920
Epoch 3/3
Epoch 3 - Average Loss: 0.0669
Average Accuracy across all labels: 0.9813
Accuracy for toxic: 0.9520
Accuracy for severe_toxic: 0.9920
Accuracy for obscene: 0.9810
Accuracy for threat: 0.9970
Accuracy for insult: 0.9740
Accuracy for identity_hate: 0.9920


In [29]:
model.save_pretrained("hate-speech-model")

In [30]:
from huggingface_hub import login

login()

In [31]:
model.push_to_hub("devesh1011/fine-tuned-bert-hate-speech-model")

adapter_model.safetensors:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/devesh1011/fine-tuned-bert-hate-speech-model/commit/5e7315c6d178964b2eb36797e2fa3084617b0a75', commit_message='Upload model', commit_description='', oid='5e7315c6d178964b2eb36797e2fa3084617b0a75', pr_url=None, repo_url=RepoUrl('https://huggingface.co/devesh1011/fine-tuned-bert-hate-speech-model', endpoint='https://huggingface.co', repo_type='model', repo_id='devesh1011/fine-tuned-bert-hate-speech-model'), pr_revision=None, pr_num=None)